In [39]:
import pandas as pd
import numpy as np
import vectorbt as vbt

In [40]:
tlt = vbt.YFData.download(
    "ITUB3.SA", 
    start="2014-01-01"
).get("Close").to_frame()
close = tlt.Close

In [41]:
# código cria séries do pandas com o mesmo formato da série com preços de fechamento. 
short_entries = pd.DataFrame.vbt.signals.empty_like(close)
short_exits = pd.DataFrame.vbt.signals.empty_like(close)
long_entries = pd.DataFrame.vbt.signals.empty_like(close)
long_exits = pd.DataFrame.vbt.signals.empty_like(close)

In [42]:
long_exits.head()

Date
2014-01-02 02:00:00+00:00    False
2014-01-03 02:00:00+00:00    False
2014-01-06 02:00:00+00:00    False
2014-01-07 02:00:00+00:00    False
2014-01-08 02:00:00+00:00    False
Name: Close, dtype: bool

In [43]:
'''
geramos os sinais de negociação com base no dia do mês.
'''

# Get short on the first day of the new month
short_entry_mask = ~tlt.index.tz_convert(None).to_period("M").duplicated()
short_entries.iloc[short_entry_mask] = True

short_entries.head()

Date
2014-01-02 02:00:00+00:00     True
2014-01-03 02:00:00+00:00    False
2014-01-06 02:00:00+00:00    False
2014-01-07 02:00:00+00:00    False
2014-01-08 02:00:00+00:00    False
Name: Close, dtype: bool

In [44]:
# Exit the short five days later
short_exit_mask = short_entries.shift(5).fillna(False)
short_exits.iloc[short_exit_mask] = True

# Get long 7 days prior to the end of the month
long_entry_mask = short_entries.shift(-7).fillna(False)
long_entries.iloc[long_entry_mask] = True

# Buy back on the last day of the month
long_exit_mask = short_entries.shift(-1).fillna(False)
long_exits.iloc[long_exit_mask] = True

long_exits.tail()

Date
2024-02-05 03:00:00+00:00    False
2024-02-06 03:00:00+00:00    False
2024-02-07 03:00:00+00:00    False
2024-02-08 03:00:00+00:00    False
2024-02-09 03:00:00+00:00    False
Name: Close, dtype: bool

In [45]:
pf =  vbt.Portfolio.from_signals(
    close=close,
    entries=long_entries, 
    exits=long_exits,
    short_entries=short_entries,
    short_exits=short_exits,  
    freq="1d"
)

pf.stats()

Start                         2014-01-02 02:00:00+00:00
End                           2024-02-09 03:00:00+00:00
Period                               2513 days 00:00:00
Start Value                                       100.0
End Value                                     31.291886
Total Return [%]                             -68.708114
Benchmark Return [%]                         225.331185
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              76.445327
Max Drawdown Duration                2102 days 00:00:00
Total Trades                                        243
Total Closed Trades                                 243
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  48.971193
Best Trade [%]                                19.892296
Worst Trade [%]                              -16

In [46]:
pf =  vbt.Portfolio.from_signals(
    close=close,
    entries=short_entries,
    exits=short_exits,
    short_entries=long_entries,
    short_exits=long_exits,
    freq="1d"
)

pf.stats()

Start                         2014-01-02 02:00:00+00:00
End                           2024-02-09 03:00:00+00:00
Period                               2513 days 00:00:00
Start Value                                       100.0
End Value                                     197.62887
Total Return [%]                               97.62887
Benchmark Return [%]                         225.331185
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              39.367265
Max Drawdown Duration                 899 days 00:00:00
Total Trades                                        243
Total Closed Trades                                 243
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  50.205761
Best Trade [%]                                16.831242
Worst Trade [%]                              -19